In [ ]:
#Processing raw MiSeq reads:
Primers used:
FishCOI-PS1-F  
CGTATTTGGYGCYTGRGCCGGRATAGT 
FishCOI-PS1-R 
CARAARCTYATRTTRTTYATTCG
#Process with dada2 pipeline, script 01_dada2_processing_COI
#In general, poor run quality: lots of reads (~90% per sample) are just primer dimers, after removal of singletons and doubletons 839 ASVs remain


In [ ]:
#Classify with RDP COI classifier v5.1.0
rdp_classifier -Xmx10g classify -t RDP_COIv5.1.0/mydata/rRNAClassifier.properties -o RDP_COI.out ASV.fna
#Filter output: Only retain phylum Actinopteri with at least 0.7 bootstrap support=> 10 ASVs remain
#Double-check ASVs compared to Comprehensive list of Canadian Freshwater fishes (Myler MSC): 9 exists in Canada, exception Desert Sucker > COI_fish_tax.txt and filtered_ASV.txt

In [ ]:
#Assign taxonomy via blast to midori
#remove spaces, commas, tabs from fasta headers
sed -e 's/\t/_/g' MIDORI2_UNIQ_SP_NUC_GB256_CO1_RAW3.fasta > MIDORI2_UNIQ_SP_NUC_GB256_CO1_RAW4.fasta
#make blast db
makeblastdb -in MIDORI2_UNIQ_SP_NUC_GB256_CO1_RAW4.fasta -dbtype nucl
#blast
blastn -db MIDORI2_UNIQ_SP_NUC_GB256_CO1_RAW4.fasta -query ASV.fna -word_size 11 -max_target_seqs 1 -outfmt 6 -evalue 1e04 -out Midori2_uniq_results.txt

In [ ]:
#Extract relevant seqs with seqtk
seqtk subseq ASV.fna filtered_ASV.list > filtered_ASV.fa

In [ ]:
# filter blast output
awk -F "\t" '{if($3>95) {print }}' Midori2_uniq_results.txt > Midori2_uniq_results_filtered.txt#533 ASVs remain, 268 are Actinopteri, but man of those are 100ID matches to tropical species etc.
#solution: also filter by match length: most questionable matches are perfect, but only 20-30bp long, real matches are 199 bp: 10 ASV remain, identical to rdp matches

In [ ]:
#Extract relevant seqs with seqtk
seqtk subseq ASV.fna filtered_ASV.list > filtered_ASV.fa

In [ ]:
#Pseudogene filter
java -jar ~/miniconda3/share/macse-2.07-0/macse_v2.07.jar -prog enrichAlignment -align ..//Midori_chordata.fa -gc_def 5 -maxSTOP_inSeq 0 -output_only_added_seq_ON -maxDEL_inSeq 5 -maxFS_inSeq 0 -maxINS_inSeq 0 -fixed_alignment_ON -out_AA MID_vert_full_align.faa -seq filtered_ASV.fa
java -jar ~/miniconda3/share/macse-2.07-0/macse_v2.07.jar -prog enrichAlignment -align ..//Midori_invert.fa -gc_def 5 -maxSTOP_inSeq 0 -output_only_added_seq_ON -maxDEL_inSeq 5 -maxFS_inSeq 0 -maxINS_inSeq 0 -fixed_alignment_ON -out_AA MID_invert_full_align.faa -seq filtered_ASV.fa
#weird: none of the good fish ASVs gets added to any alignment, and only a handful of ASVs that get added to the alignment from the full ASV files don't make a lot of sense (Bacteria, Antilope)